**Name: Mahdi Saieedi**

**Student Number: 401207254**




In this notebook, you start by loading a trained Resnet model on CIFAR10 dataset. Then, you try to attack it with an $L_{2}$ C&W attack. Next, you work to make this model better at defending using defensive distillation, and test it against two different modes of $L_{2}$ C&W attacks again.

It is recommended to use google colab to do this homework. You can connect to your drive using the code below to use it to save and load your trained models:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load CIFAR10 data

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils
import torchvision.transforms as transforms
from torchvision.models import resnet18
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

batch_size = 128

########################## Problem 1 (2  points) ###############################
# todo: Define your data loaders for training and testing                      #
################################################################################

from torch.utils.data import DataLoader

# Define your data loaders for training and testing
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

################################ End ###########################################

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

100%|██████████| 170498071/170498071 [00:02<00:00, 77126813.60it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Model set-up

In [5]:
class resnet(nn.Module):
  def __init__(self, num_cls, T=1):
    super().__init__()
    self.conv = nn.Sequential(
        *list(resnet18(weights=None).children())[:-2])

    self.fc = nn.Linear(512, num_cls)
    self.temp = T

  def forward(self, x, T=None):
    if T is None:
      T = self.temp
    x = self.conv(x)
    x = torch.flatten(x, start_dim=1)
    logits = self.fc(x)
    output = torch.softmax(logits / T, dim=1)

    return logits, output

In [6]:
# Download trained weights of the Resnet18 model on CIFAR10 dataset
!gdown 1KU4jWAwZIq0TUujAsgimLxGWUvwIEfyB

Downloading...
From (original): https://drive.google.com/uc?id=1KU4jWAwZIq0TUujAsgimLxGWUvwIEfyB
From (redirected): https://drive.google.com/uc?id=1KU4jWAwZIq0TUujAsgimLxGWUvwIEfyB&confirm=t&uuid=3c0fc021-2959-4141-8939-558c8a53f7fd
To: /content/resnet18_cifar10_model_pretrained.pth
100% 44.8M/44.8M [00:01<00:00, 32.4MB/s]


In [7]:
# load trained Resnet18 model on CIFAR10 dataset
model = resnet(len(classes)).to(device)
model_name = "resnet18_cifar10_model_pretrained.pth"
model_PATH = "/content/" + model_name
state_dict = torch.load(model_PATH, map_location=device)
model.load_state_dict(state_dict)
model = model.to(device)

# Computing clean accuracy

In [8]:
def standard_test(model, loader, device=device):
  correct = 0
  total = 0
  ########################## Problem 2 (4 points) ##############################
  # todo: Iterate over loader, compute the output and predicted                #
  # label, and update "correct" and "total" counters accordingly.              #
  ##############################################################################
  model.eval()

  with torch.no_grad():
    for data in loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      output = model(images)
      # Check if the model returns a tuple or a single tensor
      if isinstance(output, tuple):
        outputs, _ = output  # Unpack if the output is a tuple
      else:
        outputs = output  # Directly use the output if it's a tensor

      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  ################################ End #########################################
  print(f'\n Clean accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [9]:
standard_test(model, test_loader)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



 Clean accuracy of the network on the 10000 test images: 75 %


# C&W L2 Attack on the base model

Note that for the sake of simplicity, you can ignore binary search for constant c for your implementation. However, implementing it would earn you a bonus!

In [10]:

import torch.nn.functional as F
def cw_l2_attack(model, image, label, mode='outputs', targeted=False, c=1e-4, kappa=0, max_iter=1000, learning_rate=0.01):

    image = image.to(device)
    label = label.to(device)
    ######################### Problem 3 (8 points) #############################
    # todo: Implement L2 C&W attack in the two following modes:                #
    # 'outputs': use an f function which utilizes output probs of the model    #
    # 'logits': use an f function which utilizes logits of the model           #
    # Output of this function must be the resulting adversarial image.         #
    ############################################################################

    # Setup the initial perturbation
    w = torch.zeros_like(image).to(device)
    w.requires_grad = True

    # Setup optimizer
    optimizer = optim.Adam([w], lr=learning_rate)

    # Define the original image variable for perturbation adjustment
    new_img = torch.tanh(w + image) / 2  # This squashes the w + image into the [0,1] range

    for i in range(max_iter):
        optimizer.zero_grad()

        # Generate adversarial image
        adv_img = torch.tanh(w + image) / 2  # Ensure the adversarial image is within valid range

        # Pass the adversarial image through the model
        outputs = model(adv_img)

        if mode == 'outputs':
            # Use softmax probabilities
            outputs = F.softmax(outputs, dim=1)
        # If mode is 'logits', outputs are already logits

        # Calculate the f-function
        real = torch.max(outputs * label, dim=1)[0]  # Correct class probability or logit
        other = torch.max((1 - label) * outputs - label * 10000, dim=1)[0]  # Max non-target class prob or logit

        if targeted:
            # If targeted, try to make the target class most likely
            f = torch.clamp(other - real + kappa, min=0)
        else:
            # If untargeted, try to make the correct class less likely
            f = torch.clamp(real - other + kappa, min=0)

        # Calculate the CW loss
        loss = torch.sum(f) + c * torch.sum(torch.pow(w, 2))  # L2 loss term to ensure minimal perturbation

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Early stopping if achieved required confidence
        if f.item() < 1e-6:
            break

    # Create final adversarial image
    adv_img = torch.tanh(w + image) / 2

    return adv_img


    ################################ End #######################################

In [11]:
def attack_test(model, attack_model, loader, mode='outputs', targeted=False, c=0.1,  device=device):
  correct = 0
  total = 0
  ########################## Problem 4 (4 points) ##############################
  # todo: Iterate over the FIRST batch of the loader                           #
  # todo: Find an adversarial example by L2 C&W attack on attack_model         #
  # todo: Compute the output and predicted label, and updated "correct" and    #
  # "total" counters accordingly.                                              #
  ##############################################################################
  # Ensure the model is in evaluation mode
  model.eval()

  # Iterate only over the first batch of the DataLoader
  first_batch = next(iter(loader))
  images, labels = first_batch
  images, labels = images.to(device), labels.to(device)

  for i in range(images.size(0)):
    # Generate adversarial example using the C&W L2 attack
    adv_image = cw_l2_attack(attack_model, images[i].unsqueeze(0), labels[i].unsqueeze(0), mode=mode, targeted=targeted, c=c)

    # Pass the adversarial image through the model
    output = model(adv_image)
    if isinstance(outputs, tuple):
      outputs = outputs[0]

    if mode == 'outputs':
      outputs = torch.softmax(outputs, dim=1)

    # Calculate the predicted label
    _, predicted = torch.max(output.data, 1)

    # Update correct and total counters
    total += labels[i].size(0)
    correct += (predicted == labels[i]).sum().item()
  ################################ End #########################################
  print(f'\n Accuracy of the network on the 64 test images of the first batch of testloader after attacking : {100 * correct // total} %')

In [12]:
# print('C&W L2 attack on the model using its outpus(probs):')
# attack_test(model, model, test_loader, mode='outputs', device=device)


In [13]:

# print('\n C&W L2 attack on the model using its logits:')
# attack_test(model, model, test_loader, mode='logits', device=device)

# Defensive distillation

## Teacher model training

In [23]:
!pip install git+https://github.com/NightMachinery/PyNight.git
!pip install --no-deps --force-reinstall git+https://github.com/NightMachinery/PyNight.git

  Cloning https://github.com/NightMachinery/PyNight.git to /tmp/pip-req-build-b7xxpbwt
  Running command git clone --filter=blob:none --quiet https://github.com/NightMachinery/PyNight.git /tmp/pip-req-build-b7xxpbwt
  Resolved https://github.com/NightMachinery/PyNight.git to commit 209dc31f78405e1f15e77546a794a8e1e1c97689
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.8/219.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.

In [24]:
from pynight.common_torch import (
    img_tensor_show,
    TorchModelMode,
    model_device_get,
    torch_shape_get,
)

In [25]:
from sklearn.metrics import f1_score, accuracy_score

def evaluate(model, testloader, prefix=""):
    with TorchModelMode(model, 'eval'):
        device = model_device_get(model)

        predictions = []
        labels = []

        with torch.no_grad():
            for data in testloader:
                inputs, targets = data
                inputs = inputs.to(device)
                targets = targets.to(device)

                res = model(inputs)
                _, predicted = torch.max(res.logits.data, 1)

                predictions.extend(predicted.cpu().numpy())
                labels.extend(targets.cpu().numpy())

        accuracy = accuracy_score(labels, predictions)

        f1 = f1_score(labels, predictions, average='weighted')
        print('{}Accuracy: {:.2f}%, F1 Score: {:.2f}'.format(prefix, accuracy * 100, f1))


In [20]:
def standard_train(
    model, loader, num_epoch, optimizer, criterion, T=None, device=device
):
    ###################### Problem 2 (4 points) ##################################
    # todo: Iterate over loader in each epoch                                    #
    # todo: Compute the model's output for each batch at the given temperature T #
    # todo: Compute the loss function and take a step by the optimizer           #
    # todo: Monitor the training procedure                                       #
    ##############################################################################

    evaluate(model=model, testloader=test_loader, prefix="before training: ")

    for epoch in range(1, num_epoch + 1):
        running_loss = 0.0
        for i, data in enumerate(loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            res = model(inputs, T=T)

            loss = criterion(res.logits_scaled, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch}, Loss: {running_loss / len(loader)}")

        if epoch % 5 == 0:
            evaluate(model=model, testloader=test_loader, prefix=f"Epoch {epoch}: ")

In [26]:
T = 100
teacher = resnet(len(classes), T=T).to(device)
teacher_optim = optim.Adam(teacher.parameters())
teacher_criterion = nn.CrossEntropyLoss()

In [17]:
# # load teacher model (if you already trained it)
# model_name = "teacherModel.pth"
# teacher_model_PATH = "/content/drive/MyDrive/" + model_name
# state_dict = torch.load(teacher_model_PATH)
# teacher.load_state_dict(state_dict)
# teacher = teacher.to(device)

In [ ]:
standard_train(model=teacher,
            loader=train_loader,
            num_epoch=15,
            optimizer=teacher_optim,
            criterion=teacher_criterion,
            device=device)

In [ ]:
# save teacher model (only if you just trained it)
teacher.eval()
model_name = "teacherModel.pth"
teacher_model_PATH = "/content/drive/MyDrive/" + model_name
torch.save(teacher.state_dict(), teacher_model_PATH)

## Student model training

In [29]:
def distillation(teacher, student, loader, num_epoch, optimizer, criterion, device=device):
  ########################## Problem 5 (6 points) ##############################
  # todo: Iterate over loader in each epoch                                    #
  # todo: Compute MSE loss between student's logit and teacher's logit         #
  # todo: Take a step by the optimizer                                         #
  # todo: Monitor the training procedure                                       #
  ##############################################################################
  evaluate(model=student, testloader=testloader, prefix="before training: ")
  for epoch in range(1, num_epoch + 1):
      running_loss = 0.0
      for i, data in enumerate(loader, 0):
          inputs, labels = data[0].to(device), data[1].to(device)

          optimizer.zero_grad()

          res = student(inputs, T=T)

          with torch.no_grad():
              teacher_res = teacher(inputs, T=T)
              if mode == 'logits':
                  teacher_labels = teacher_res.logits_scaled
              else:
                  teacher_labels = teacher_res.output

          loss = criterion(res.logits_scaled, teacher_labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      print(f"Epoch {epoch}, Loss: {running_loss / len(loader)}")

      if epoch % 5 == 0:
          evaluate(model=student, testloader=testloader, prefix=f"Epoch {epoch}: ")
  ################################ End #########################################

In [30]:
T = 100
student = resnet(len(classes), T=T).to(device)
student_optim = optim.Adam(student.parameters())
std_criterion = nn.MSELoss()

In [ ]:
# load student model (only if you already trained it)
model_name = "studentModel.pth"
student_model_PATH = "/content/drive/MyDrive/" + model_name
state_dict = torch.load(student_model_PATH)
student.load_state_dict(state_dict)
student = student.to(device)

In [ ]:
distillation(teacher=teacher,
             student=student,
             loader=train_loader,
             num_epoch=15,
             optimizer=student_optim,
             criterion=std_criterion,
             device=device)

In [ ]:
# save student model (only if you just trained it)
student.eval()
model_name = "studentModel.pth"
teacher_model_PATH = "/content/drive/MyDrive/" + model_name
torch.save(student.state_dict(), teacher_model_PATH)

# C&W L2 Attack on the distilled  model

In [ ]:
print('C&W L2 attack on the student model using its outpus(probs):')
attack_test(student, student, test_loader, mode='outputs', device=device)

print('\n C&W L2 attack on the student model using its logits:')
attack_test(student, student, test_loader, mode='logits', device=device)

# Analyzing the results

Answer the following questions below them: (6 points)

1. Why do you think the attack success rate of the logits mode is generally better than the outputs(probs) mode?
#1. Direct Access to Model Decision Boundaries
#2. Gradient Sensitivity and Saturation
#3. Effectiveness in Targeting Specific Outcomes
#4. Optimization Stability

#######################################################################################


2. Why defensive distillation completely failed in the logits mode?

fails in logits mode because logits are unbounded and provide more gradient information. This allows attackers to exploit larger gradient signals for crafting adversarial examples, bypassing the distillation's intended effect of smoothing out model responses


#######################################################################################
3. As you have seen, defensive distillation is a mirage. Do you think that there is still some scenarios that this defense may be helpful?

t can enhance model generalization and defend against simpler attacks like FGSM by smoothing model outputs and reducing sensitivity to small input variations. Distillation may also improve model performance on noisy or slightly perturbed data, acting as a form of regularization.

